In [1]:
from pathlib import Path
import requests
import pickle
import gzip
from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import math

import RNN_model

In [2]:
DATA_PATH = Path("data")
PATH = DATA_PATH / "mnist"

PATH.mkdir(parents=True, exist_ok=True)

URL = "http://deeplearning.net/data/mnist/"
FILENAME = "mnist.pkl.gz"

if not (PATH / FILENAME).exists():
        content = requests.get(URL + FILENAME).content
        (PATH / FILENAME).open("wb").write(content)

In [3]:
with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")

x_train, y_train, x_valid, y_valid = map(
    torch.tensor, (x_train.reshape(x_train.shape[0],784,1), y_train, x_valid.reshape(x_valid.shape[0],784,1), y_valid)
)

train_ds = TensorDataset(x_train, y_train)
n, c, temp = x_train.shape
bs = 64  # batch size
train_dl = DataLoader(train_ds, batch_size=bs)

valid_ds = TensorDataset(x_valid, y_valid)
valid_dl = DataLoader(valid_ds, batch_size=100, shuffle=True)

In [4]:
n_steps = 28*28
n_inputs = 1
n_neurons = 200
n_outputs = 10
loss_func = F.cross_entropy
opt_func = optim.Adam
lr = 0.5

model = RNN_model.RNN_classifier.get_model(n_steps, n_inputs, n_neurons, n_outputs, loss_func, opt_func, lr)

In [5]:
print(model.accuracy(valid_dl))

tensor(0.1400)


In [ ]:
loss_array = model.fit(1,train_dl)

plt.plot(loss_array)

print(model.accuracy(valid_dl))